# 환율 라벨링 후 ngram과 merge 하기

In [2]:
import pandas as pd

In [3]:
news = pd.read_csv('C:/Users/user/Desktop/globo/news_ngram.csv')

In [21]:
news

,date,content,main_title,sub_title,total,unigram,bigram,trigram,fourgram,fivegram
0,2018-06-01,Em um último movimento para tentar ressuscitar...,Áudios convocam caminhoneiros para novo ato,"Em vídeos, um dos líderes do movimento pede qu...",Áudios convocam caminhoneiros para novo atoEm ...,"nã,páti,descans,local,pass,noit,famíl,viag,lis...","nã;páti,páti;descans,descans;local,local;pass,...","nã;páti;descans,páti;descans;local,descans;loc...","nã;páti;descans;local,páti;descans;local;pass,...","nã;páti;descans;local;pass,páti;descans;local;..."
1,2018-06-01,As ações da Petrobras fecharam em queda de qua...,"Ações da Petrobras caem quase 15%, e estatal p...",Recibos de ações negociadas nos Estados Unidos...,"Ações da Petrobras caem quase 15%, e estatal p...","o,banc,central,mantev,atuaçã,merc,câmbi,vend,1...","o;banc,banc;central,central;mantev,mantev;atua...","o;banc;central,banc;central;mantev,central;man...","o;banc;central;mantev,banc;central;mantev;atua...","o;banc;central;mantev;atuaçã,banc;central;mant..."
2,2018-06-01,O Ministério do Trabalho suspendeu por 30 dias...,"Após operação da PF, ministério suspende regis...",Decisão paralisa análises dos registros. Deput...,"Após operação da PF, ministério suspende regis...","indic,aind,investig,trat,pagament,propin,minis...","indic;aind,aind;investig,investig;trat,trat;pa...","indic;aind;investig,aind;investig;trat,investi...","indic;aind;investig;trat,aind;investig;trat;pa...","indic;aind;investig;trat;pagament,aind;investi..."
3,2018-06-01,"Depois de , pode ser o da , gigante de alime...","Após pedir demissão, Parente pode ser o novo p...",Parente assumiu recentemente o comando do cons...,"Após pedir demissão, Parente pode ser o novo p...","a,empres,possu,hoj,14,pesso,—entr,vice-preside...","a;empres,empres;possu,possu;hoj,hoj;14,14;pess...","a;empres;possu,empres;possu;hoj,possu;hoj;14,h...","a;empres;possu;hoj,empres;possu;hoj;14,possu;h...","a;empres;possu;hoj;14,empres;possu;hoj;14;pess..."
4,2018-06-01,O conselho de administração da Petrobras indic...,Petrobras confirma Ivan Monteiro como novo pre...,"Monteiro, atual diretor financeiro, está a cam...",Petrobras confirma Ivan Monteiro como novo pre...,"vei,bendin,banc,brasil,onde,cuid,áre,financeir...","vei;bendin,bendin;banc,banc;brasil,brasil;onde...","vei;bendin;banc,bendin;banc;brasil,banc;brasil...","vei;bendin;banc;brasil,bendin;banc;brasil;onde...","vei;bendin;banc;brasil;onde,bendin;banc;brasil..."
...,...,...,...,...,...,...,...,...,...,...
9527,2019-12-31,"O presidente dos Estados Unidos, Donald Trump,...",Trump anuncia que assinará o acordo comercial ...,Presidente dos Estados Unidos disse que irá ao...,Trump anuncia que assinará o acordo comercial ...,"ele,indic,viaj,pequim,dat,posterior,fas,dois,d...","ele;indic,indic;viaj,viaj;pequim,pequim;dat,da...","ele;indic;viaj,indic;viaj;pequim,viaj;pequim;d...","ele;indic;viaj;pequim,indic;viaj;pequim;dat,vi...","ele;indic;viaj;pequim;dat,indic;viaj;pequim;da..."
9528,2019-12-31,O governo informou que transferiu nesta terça-...,"Governo transfere R$ 11,7 bilhões a estados e ...",Equipe econômica espera avançar com o pacto fe...,"Governo transfere R$ 11,7 bilhões a estados e ...","em,cas,licit,empres,vencedor,oferec,uniã,maior...","em;cas,cas;licit,licit;empres,empres;vencedor,...","em;cas;licit,cas;licit;empres,licit;empres;ven...","em;cas;licit;empres,cas;licit;empres;vencedor,...","em;cas;licit;empres;vencedor,cas;licit;empres;..."
9529,2019-12-31,"O presidente dos Estados Unidos, , disse nesta...",Trump diz que acordo comercial EUA-China será ...,Fase 1 do tratado será celebrada na Casa Branc...,Trump diz que acordo comercial EUA-China será ...,"trump,anunc,primeir,vez,plan,pact,comercial,in...","trump;anunc,anunc;primeir,primeir;vez,vez;plan...","trump;anunc;primeir,anunc;primeir;vez,primeir;...","trump;anunc;primeir;vez,anunc;primeir;vez;plan...","trump;anunc;primeir;vez;plan,anunc;primeir;vez..."
9530,2019-12-31,"O presidente do Supremo Tribunal Federal, mini

### 환율 라벨링

In [4]:
currency = pd.read_csv('C:/Users/user/Desktop/환율.csv', encoding='euc-kr')
currency

,날짜,종가,오픈,고가,저가,변동 %
0,2020년 01월 01일,287.35,287.37,287.49,287.16,-0.01%
1,2019년 12월 31일,287.37,288.10,288.40,286.90,-0.25%
2,2019년 12월 30일,288.09,287.06,288.79,285.55,0.51%
3,2019년 12월 27일,286.63,286.53,287.45,285.50,0.02%
4,2019년 12월 26일,286.56,284.29,287.25,284.12,0.82%
...,...,...,...,...,...,...
409,2018년 06월 07일,274.38,277.14,278.35,270.64,-1.03%
410,2018년 06월 06일,277.24,281.50,281.73,276.66,-1.54%
411,2018년 06월 05일,281.58,285.79,286.50,280.73,-1.45%
412,2018년 06월 04일,285.71,284.51,287.35,283.65,0.32%


날짜 형태에 맞게 변환

In [48]:
currency['날짜'] = currency['날짜'].str.replace('년 ','-')
currency['날짜'] = currency['날짜'].str.replace('월 ','-')
currency['날짜'] = currency['날짜'].str.replace('일','')
currency['날짜']=pd.to_datetime(currency['날짜'],format="%Y-%m-%d")

In [59]:
ls = []
for i in currency['변동 %']:
    if len(i) == 5:
        ls.append('+'+i)
    else:
        ls.append(i)

In [69]:
a = pd.DataFrame(ls, columns=['변동'])
real = pd.concat([currency, a], axis=1)

,날짜,종가,오픈,고가,저가,변동 %,변동
0,2020-01-01,287.35,287.37,287.49,287.16,-0.01%,-0.01%
1,2019-12-31,287.37,288.10,288.40,286.90,-0.25%,-0.25%
2,2019-12-30,288.09,287.06,288.79,285.55,0.51%,+0.51%
3,2019-12-27,286.63,286.53,287.45,285.50,0.02%,+0.02%
4,2019-12-26,286.56,284.29,287.25,284.12,0.82%,+0.82%
...,...,...,...,...,...,...,...
409,2018-06-07,274.38,277.14,278.35,270.64,-1.03%,-1.03%
410,2018-06-06,277.24,281.50,281.73,276.66,-1.54%,-1.54%
411,2018-06-05,281.58,285.79,286.50,280.73,-1.45%,-1.45%
412,2018-06-04,285.71,284.51,287.35,283.65,0.32%,+0.32%


In [72]:
real['변동'] = real['변동'].str[:5]
real = real.drop(['종가', '오픈', '고가','저가', '변동 %'], axis=1)
real['변동']=pd.to_numeric(real['변동'])

In [111]:
# 라벨링 칼럼 추가
real['label']=''

In [112]:
for i in range(len(real)):
    if real['변동'][i] >= 0.4:
        real['label'][i] = real['label'][i].replace('', '1')
    elif real['변동'][i] <= -0.4:
        real['label'][i] = real['label'][i].replace('', '-1')
    else:
        real['label'][i] = real['label'][i].replace('', '0')


<ipython-input-112-de7073b1d3b4>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  real['label'][i] = real['label'][i].replace('', '0')
<ipython-input-112-de7073b1d3b4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  real['label'][i] = real['label'][i].replace('', '1')
<ipython-input-112-de7073b1d3b4>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  real['label'][i] = real['label'][i].replace('', '-1')


### 뉴스와 merge

In [134]:
real.to_csv('C:/Users/user/Desktop/globo/real_krw.csv', index=False)

In [5]:
real = pd.read_csv('C:/Users/user/Desktop/globo/real_krw.csv')

In [6]:
real.rename(columns = {'날짜' : 'date'}, inplace = True)

In [7]:
news_label = pd.merge(news,real,on='date',how='inner')
news_label.drop(['content', 'main_title', 'sub_title', 'total', '변동'], axis='columns', inplace=True)

In [8]:
news_label

,date,unigram,bigram,trigram,fourgram,fivegram,label
0,2018-06-01,"nã,páti,descans,local,pass,noit,famíl,viag,lis...","nã;páti,páti;descans,descans;local,local;pass,...","nã;páti;descans,páti;descans;local,descans;loc...","nã;páti;descans;local,páti;descans;local;pass,...","nã;páti;descans;local;pass,páti;descans;local;...",-1
1,2018-06-01,"o,banc,central,mantev,atuaçã,merc,câmbi,vend,1...","o;banc,banc;central,central;mantev,mantev;atua...","o;banc;central,banc;central;mantev,central;man...","o;banc;central;mantev,banc;central;mantev;atua...","o;banc;central;mantev;atuaçã,banc;central;mant...",-1
2,2018-06-01,"indic,aind,investig,trat,pagament,propin,minis...","indic;aind,aind;investig,investig;trat,trat;pa...","indic;aind;investig,aind;investig;trat,investi...","indic;aind;investig;trat,aind;investig;trat;pa...","indic;aind;investig;trat;pagament,aind;investi...",-1
3,2018-06-01,"a,empres,possu,hoj,14,pesso,—entr,vice-preside...","a;empres,empres;possu,possu;hoj,hoj;14,14;pess...","a;empres;possu,empres;possu;hoj,possu;hoj;14,h...","a;empres;possu;hoj,empres;possu;hoj;14,possu;h...","a;empres;possu;hoj;14,empres;possu;hoj;14;pess...",-1
4,2018-06-01,"vei,bendin,banc,brasil,onde,cuid,áre,financeir...","vei;bendin,bendin;banc,banc;brasil,brasil;onde...","vei;bendin;banc,bendin;banc;brasil,banc;brasil...","vei;bendin;banc;brasil,bendin;banc;brasil;onde...","vei;bendin;banc;brasil;onde,bendin;banc;brasil...",-1
...,...,...,...,...,...,...,...
8524,2019-12-31,"ele,indic,viaj,pequim,dat,posterior,fas,dois,d...","ele;indic,indic;viaj,viaj;pequim,pequim;dat,da...","ele;indic;viaj,indic;viaj;pequim,viaj;pequim;d...","ele;indic;viaj;pequim,indic;viaj;pequim;dat,vi...","ele;indic;viaj;pequim;dat,indic;viaj;pequim;da...",0
8525,2019-12-31,"em,cas,licit,empres,vencedor,oferec,uniã,maior...","em;cas,cas;licit,licit;empres,empres;vencedor,...","em;cas;licit,cas;licit;empres,licit;empres;ven...","em;cas;licit;empres,cas;licit;empres;vencedor,...","em;cas;licit;empres;vencedor,cas;licit;empres;...",0
8526,2019-12-31,"trump,anunc,primeir,vez,plan,pact,comercial,in...","trump;anunc,anunc;primeir,primeir;vez,vez;plan...","trump;anunc;primeir,anunc;primeir;vez,primeir;...","trump;anunc;primeir;vez,anunc;primeir;vez;plan...","trump;anunc;primeir;vez;plan,anunc;primeir;vez...",0
8527,2019-12-31,"toffol,conced,limin,estar,plantã,durant,recess...","toffol;conced,conced;limin,limin;estar,estar;p...","toffol;conced;limin,conced;limin;estar,limin;e...","toffol;conced;limin;estar,conced;limin;estar;p...","toffol;conced;limin;estar;plantã,conced;limin;...",0


In [9]:
news_label['ngram']=news_label['unigram']+news_label['bigram']+news_label['trigram']+news_label['fourgram']+news_label['fivegram']
df = news_label.drop(['unigram', 'bigram','trigram', 'fourgram', 'fivegram'], axis=1)
df

,date,label,ngram
0,2018-06-01,-1,"nã,páti,descans,local,pass,noit,famíl,viag,lis..."
1,2018-06-01,-1,"o,banc,central,mantev,atuaçã,merc,câmbi,vend,1..."
2,2018-06-01,-1,"indic,aind,investig,trat,pagament,propin,minis..."
3,2018-06-01,-1,"a,empres,possu,hoj,14,pesso,—entr,vice-preside..."
4,2018-06-01,-1,"vei,bendin,banc,brasil,onde,cuid,áre,financeir..."
...,...,...,...
8524,2019-12-31,0,"ele,indic,viaj,pequim,dat,posterior,fas,dois,d..."
8525,2019-12-31,0,"em,cas,licit,empres,vencedor,oferec,uniã,maior..."
8526,2019-12-31,0,"trump,anunc,primeir,vez,plan,pact,comercial,in..."
8527,2019-12-31,0,"toffol,conced,limin,estar,plantã,durant,recess..."


In [10]:
df.to_csv('C:/Users/user/Desktop/globo/ngram_label.csv', index=False)